<a href="https://colab.research.google.com/github/adityab-28/WTurbine_FS_based_CM/blob/main/Trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('drive/MyDrive/new/T01_1009.csv')

In [ ]:
data.head()

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Hyd_Oil_Temp_Avg,Gear_Oil_Temp_Avg,Gear_Bear_Temp_Avg,Nac_Temp_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Avg,Amb_WindSpeed_Std,Amb_WindDir_Relative_Avg,Amb_WindDir_Abs_Avg,Amb_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ActPwrGen2,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_ReactPwrGen0,Prod_LatestAvg_ReactPwrGen1,Prod_LatestAvg_ReactPwrGen2,Prod_LatestAvg_TotReactPwr,HVTrafo_Phase1_Temp_Avg,HVTrafo_Phase2_Temp_Avg,HVTrafo_Phase3_Temp_Avg,Grd_InverterPhase1_Temp_Avg,Cont_Top_Temp_Avg,Cont_Hub_Temp_Avg,Cont_VCP_Temp_Avg,Gen_SlipRing_Temp_Avg,...,Blds_PitchAngle_Avg,Blds_PitchAngle_Std,Cont_VCP_ChokcoilTemp_Avg,Grd_RtrInvPhase1_Temp_Avg,Grd_RtrInvPhase2_Temp_Avg,Grd_RtrInvPhase3_Temp_Avg,Cont_VCP_WtrTemp_Avg,Grd_Prod_Pwr_Avg,Grd_Prod_CosPhi_Avg,Grd_Prod_Freq_Avg,Grd_Prod_VoltPhse1_Avg,Grd_Prod_VoltPhse2_Avg,Grd_Prod_VoltPhse3_Avg,Grd_Prod_CurPhse1_Avg,Grd_Prod_CurPhse2_Avg,Grd_Prod_CurPhse3_Avg,Grd_Prod_Pwr_Max,Grd_Prod_Pwr_Min,Grd_Busbar_Temp_Avg,Rtr_RPM_Std,Amb_WindSpeed_Est_Avg,Grd_Prod_Pwr_Std,Grd_Prod_ReactPwr_Avg,Grd_Prod_ReactPwr_Max,Grd_Prod_ReactPwr_Min,Grd_Prod_ReactPwr_Std,Grd_Prod_PsblePwr_Avg,Grd_Prod_PsblePwr_Max,Grd_Prod_PsblePwr_Min,Grd_Prod_PsblePwr_Std,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,T01,2016-01-01T00:00:00+00:00,1277.4,1226.1,1249.0,9.0,41,58,59,58,30,44,48,28,11.3,10.9,11.1,11.6,0.5,3.3,0.9,-12.4,206.1,18,-107,4420,0,4313,-99,-5636,0,-5735,68,76,65,39,39,28,43,25,...,0.6,0.9,91,39,39,38,39,26.2,0.7,50.0,401.6,399.9,399.2,46.5,61.1,45.5,118.4,-33.5,38,0.1,3.6,35.3,-34.9,13.2,-100.0,36.2,29.4,119.3,0.0,31.9,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37,218.5
1,T01,2016-01-01T00:10:00+00:00,1268.3,93.3,999.7,435.9,41,58,59,59,30,44,48,28,11.2,0.0,8.8,12.5,0.4,3.2,0.9,-1.1,217.4,18,-753,2488,0,1735,-867,-3871,0,-4738,68,76,65,38,39,28,42,25,...,8.1,14.7,91,37,36,37,37,10.1,0.7,50.0,401.9,400.7,399.5,34.3,41.9,33.6,101.5,-38.2,38,4.0,3.3,30.2,-28.0,1.8,-100.1,34.6,21.7,102.3,0.0,24.7,-79.6,0.0,-501.0,123.7,79.6,501.0,0.0,123.7,37,218.5
2,T01,2016-01-01T00:20:00+00:00,1394.4,220.3,774.0,486.1,41,57,57,57,30,43,46,29,12.3,1.9,6.8,8.8,0.7,4.3,1.0,-5.8,207.4,18,-784,10488,0,9704,-410,-1865,0,-2275,68,76,65,39,39,28,42,25,...,10.4,12.4,87,38,37,37,38,59.0,0.7,50.0,401.4,400.5,398.8,78.6,92.7,86.1,219.4,-34.1,38,4.3,4.1,77.5,-13.9,999.9,-108.1,154.8,88.9,224.9,0.0,54.3,-314.0,0.0,-1000.0,357.4,314.0,1000.0,0.0,357.4,37,213.3
3,T01,2016-01-01T00:30:00+00:00,1306.6,1212.4,1257.1,17.0,40,56,57,57,30,44,48,28,11.5,10.7,11.1,9.9,0.7,4.4,1.1,9.6,232.0,18,0,22673,0,22673,0,-15822,0,-15822,68,76,65,39,39,28,42,25,...,-0.5,0.8,87,38,38,38,38,135.7,0.8,50.0,399.7,398.1,396.9,128.1,160.8,136.8,250.3,-27.7,38,0.1,4.6,53.9,-94.9,0.0,-105.1,18.4,135.3,261.4,0.0,53.1,-654.0,0.0,-1000.0,245.8,654.0,1000.0,0.0,245.8,36,222.4
4,T01,2016-01-01T00:40:00+00:00,1320.9,1224.1,1257.7,18.0,40,57,58,57,30,44,48,28,11.7,10.8,11.1,9.9,0.8,4.1,1.2,23.3,245.6,18,-75,16581,0,16506,-79,-13314,0,-13394,68,76,65,39,39,28,42,25,...,-0.1,1.2,88,39,38,38,39,98.9,0.7,50.0,399.5,397.9,396.8,104.7,130.3,108.3,212.5,-50.2,38,0.2,4.3,65.9,-80.3,17.8,-107.1,35.8,102.7,221.8,0.0,60.2,-496.1,0.0,-1000.0,302.7,496.1,1000.0,0.0,302.7,36,222.4


#Feature Reduction

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
timestamp = data['Timestamp']
turbine_id = data['Turbine_ID']
Y = data['Gear_Oil_Temp_Avg']
data = data.drop(['Timestamp','Turbine_ID','Gear_Oil_Temp_Avg'], axis = 1)

data.shape

**NCA(Neighbourhood Component Analysis**

In [ ]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
nca = NeighborhoodComponentsAnalysis(random_state=42, n_components=54) 
nca.fit(data, Y)

NeighborhoodComponentsAnalysis(n_components=54, random_state=42)

In [ ]:
transform_data = nca.transform(data)

In [ ]:
transform_data.shape

(1008, 54)

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import (KNeighborsClassifier, NeighborhoodComponentsAnalysis)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transform_data, Y, test_size=0.3)

#Linear Regression

**Training the linear regression model on training set**


In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

lin = regressor.fit(X_train, y_train)

**Predicting the test set results**

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
# plt.scatter(X_train, y_train, color = 'red')
# plt.plot(X_train, regressor.predict(X_train), color = 'blue')

In [ ]:
print(X_train.size)
print(y_train.size)
print(X_train.shape)
print(y_train.shape)

38070
705
(705, 54)
(705,)


In [ ]:
# y' = m.x' + c
print(regressor.coef_)
print(regressor.intercept_)

[ 3.92166109e-05  3.84258397e-04  2.89496806e-03  2.14157890e-03
 -3.05853265e-03 -1.17854253e-03 -9.91420075e-04 -1.36907039e-03
 -6.20704513e-04 -3.09384836e-04  5.59428075e-03  2.60928300e-03
 -3.00150485e-04  1.78086243e-03 -1.38465584e-02  9.08593720e-04
 -5.39111583e-03 -7.91519032e-03  2.32660575e-03  1.61248596e-02
 -9.76842818e-03 -3.09733336e-03 -6.68538603e-02 -1.81417639e-02
 -8.13786562e-03 -1.13716725e-02 -3.14406202e-02  2.09960204e-02
  1.08215842e-01 -5.10131018e-02 -1.70347232e-01 -1.36760267e-01
 -2.61542993e-02  1.72809328e-02  4.91687213e-02 -4.07778105e-02
  2.80540402e-01  1.53896669e-02 -2.02843297e-01  8.72887050e-02
  1.46112950e-02 -1.12565320e-02  9.54275923e-01 -2.94445513e-01
 -1.59649473e-01 -6.47588975e-02  3.39915805e-02  2.67705340e-02
 -4.14668865e-02  6.20148155e-02  3.50987485e-01 -1.73388790e-02
 -1.58523732e-01 -2.84818359e-01]
0.9806212757144408


In [ ]:
# X_train_norm = (X_train - X_train.mean())/X_train.std()
# y_train_norm = (y_train - y_train.mean())/y_train.std()
# plt.scatter(X_train_norm, y_train_norm, color = 'red')
# plt.plot(X_train_norm, regressor.predict(X_train_norm), color = 'blue')
regressor.score(X_test,y_test)

0.976601676036673

# Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
pred_train = reg.predict(X_train)
pred_test = reg.predict(X_test)
print("Train Score : - ", mean_squared_error(y_train, pred_train))
print("Test Score : - ", mean_squared_error(y_test, pred_test))

Train Score : -  0.0
Test Score : -  6.887788778877888


In [ ]:
print("MSE", mean_squared_error(y_test, pred_test))
print("MAE", mean_absolute_error(y_test,pred_test))
print("MAPE", mean_absolute_percentage_error(y_test, pred_test))


MSE 6.887788778877888
MAE 1.5874587458745875
MAPE 0.036619329934931874


#ANN

In [ ]:
X_train.shape, y_test.shape

((705, 54), (303,))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model=Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(54,)))
model.add(Dense(5))

model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, y_train, epochs=20, verbose=1, validation_split=0.3)

Epoch 1/20
16/16 [==============================] - 1s 13ms/step - loss: 837244544.0000 - val_loss: 374532352.0000
Epoch 2/20
16/16 [==============================] - 0s 4ms/step - loss: 268264464.0000 - val_loss: 95778488.0000
Epoch 3/20
16/16 [==============================] - 0s 4ms/step - loss: 52191780.0000 - val_loss: 9970946.0000
Epoch 4/20
16/16 [==============================] - 0s 3ms/step - loss: 4893603.0000 - val_loss: 2322831.0000
Epoch 5/20
16/16 [==============================] - 0s 4ms/step - loss: 2566403.5000 - val_loss: 1977346.8750
Epoch 6/20
16/16 [==============================] - 0s 3ms/step - loss: 1590164.3750 - val_loss: 1053113.5000
Epoch 7/20
16/16 [==============================] - 0s 3ms/step - loss: 1019603.0000 - val_loss: 796406.5000
Epoch 8/20
16/16 [==============================] - 0s 4ms/step - loss: 752610.3750 - val_loss: 547364.6250
Epoch 9/20
16/16 [==============================] - 0s 3ms/step - loss: 488261.6250 - val_loss: 322384.4062
Epoch 

In [ ]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 750.5894


750.58935546875

In [ ]:
y_pred=model.predict(Y)
plt.figure(figsize=(20, 10))
plt.plot(Y[:500], label='Y_true')
plt.plot(y_pred[:500], label='Y_pred')
plt.legend()

# LSSVR - Least Squared Support Vector Regression

In [ ]:
#!pip install  
#from lssvr import LSSVR


In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.utils import check_X_y, check_array
from sklearn.exceptions import NotFittedError
from scipy.sparse.linalg import lsmr


class LSSVR(BaseEstimator, RegressorMixin):
    """Least Squares Support Vector Regression.
    Parameters
    ----------
    C : float, default=2.0
        Regularization parameter. The strength of the regularization is
        inversely proportional to C. Must be strictly positive.
    kernel : {'linear', 'rbf'}, default='linear'
        Specifies the kernel type to be used in the algorithm.
        It must be 'linear', 'rbf' or a callable.
    gamma : float, default = None
        Kernel coefficient for 'rbf'
    Attributes
    ----------
    support_: boolean np.array of shape (n_samples,), default = None
        Array for support vector selection.
    alpha_ : array-like
        Weight matrix
    bias_ : array-like
        Bias vector
    """

    def __init__(self, C=2.0, kernel='linear', gamma=None):
        self.C = C
        self.kernel = kernel
        self.gamma = gamma

    def fit(self, X, y, support=None):
        """Fit the model according to the given training data.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training data
        y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Target values.
        support : boolean np.array of shape (n_samples,), default = None
            Array for support vector selection.
        Returns
        -------
        self : object
            An instance of the estimator.
        """

        X, y = check_X_y(X, y, multi_output=True, dtype='float')

        if not support:
            self.support_ = np.ones(X.shape[0], dtype=bool)
        else:
            self.support_ = check_array(support, ensure_2d=False, dtype='bool')

        self.support_vectors_ = X[self.support_, :]
        support_labels = y[self.support_]

        self.K_ = self.kernel_func(X, self.support_vectors_)
        omega = self.K_.copy()
        np.fill_diagonal(omega, omega.diagonal()+self.support_/self.C)

        D = np.empty(np.array(omega.shape) + 1)

        D[1:, 1:] = omega
        D[0, 0] = 0
        D[0, 1:] = 1
        D[1:, 0] = 1

        shape = np.array(support_labels.shape)
        shape[0] += 1
        t = np.empty(shape)

        t[0] = 0
        t[1:] = support_labels

        # TODO: maybe give access to  lsmr atol and btol ?
        try:
            z = lsmr(D.T, t)[0]
        except:
            z = np.linalg.pinv(D).T @ t

        self.bias_ = z[0]
        self.alpha_ = z[1:]
        self.alpha_ = self.alpha_[self.support_]

        return self

    def predict(self, X):
        """
        Predict using the estimator.
        Parameters
        ----------
        X : array-like or sparse matrix, shape (n_samples, n_features)
            Samples.
        Returns
        -------
        y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Returns predicted values.
        """

        if not hasattr(self, 'support_vectors_'):
            raise NotFittedError

        X = check_array(X, ensure_2d=False)
        K = self.kernel_func(X, self.support_vectors_)
        return (K @ self.alpha_) + self.bias_

    def kernel_func(self, u, v):
        if self.kernel == 'linear':
            return np.dot(u, v.T)

        elif self.kernel == 'rbf':
            return rbf_kernel(u, v, gamma=self.gamma)

        elif callable(self.kernel):
            if hasattr(self.kernel, 'gamma'):
                return self.kernel(u, v, gamma=self.gamma)
            else:
                return self.kernel(u, v)
        else:
            # default to linear
            return np.dot(u, v.T)

    def score(self, X, y):
        from scipy.stats import pearsonr
        p, _ = pearsonr(y, self.predict(X))
        return p ** 2

    def norm_weights(self):
        A = self.alpha_.reshape(-1, 1) @ self.alpha_.reshape(-1, 1).T

        W = A @ self.K_[self.support_, :]
        return np.sqrt(np.sum(np.diag(W)))

In [ ]:
model = LSSVR(kernel='linear')
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
print('LSSVR Linear\nMSE', mean_squared_error(y_test, y_hat))
print('R2 ',model.score(X_test, y_test))

#for Radial basis kernel
model = LSSVR(kernel='rbf', gamma=0.01)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
print('LSSVR RBF\nMSE', mean_squared_error(y_test, y_hat))
print('R2 ',model.score(X_test, y_test))
print("MAE(train) : - ", mean_absolute_error(y_train, pred_train))
print("MAE(test) : - ", mean_absolute_error(y_test, pred_test))

LSSVR Linear
MSE 15.60076814137458
R2  0.7922391115617073
LSSVR RBF
MSE 63.16235677067237
R2  0.008440906001384706
MAE(train) : -  0.0
MAE(test) : -  1.5874587458745875
